In [ ]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import webbrowser
import os
import numpy as np
from sklearn.impute import SimpleImputer
  
df = pd.read_csv("Tema_4.csv")


# Paleta de colores personalizada
colores = {
    'principal': '#0047AB',   
    'oscuro': '#00264D',      
    'cian': '#00A6ED',        
    'dorado': '#C5A100',      
    'gris': '#E5E5E5'         
}

# Aplicar estilo general
plt.style.use('default')
sns.set_palette([colores['principal']])
sns.set_context("notebook", font_scale=1.0)
plt.rcParams.update({
    'axes.facecolor': 'white',
    'axes.edgecolor': colores['oscuro'],
    'grid.color': '#cccccc',
    'grid.linestyle': '--',
    'grid.alpha': 0.6
})

color_base = colores['principal']


# Análisis Exploratorio

In [ ]:
%matplotlib inline

print(f"El dataset contiene {df.shape[0]} filas (registros) y {df.shape[1]} columnas")

filas = df.shape[0]
columnas = df.shape[1]
total_celdas = df.size

info_df = pd.DataFrame({
    'Columnas': df.columns,
    'Tipo de dato': df.dtypes,
    'Valores no nulos': df.notnull().sum(),
    'Valores nulos': df.isnull().sum(),
    'Porcentaje nulos (%)': (df.isnull().sum() / len(df) * 100).round(2)
})

display(info_df)

print("------------------------- VALORES NULOS DEL DATASET ------------------------")
porcentaje_nulos = (df.isna().mean() * 100).sort_values()

plt.figure(figsize=(10,6))
plt.barh(porcentaje_nulos.index, porcentaje_nulos.values, color=color_base, edgecolor='black')
plt.xlabel('Porcentaje de valores nulos (%)')
plt.ylabel('Variables')
plt.title('Porcentaje de valores nulos por variable')

for i, v in enumerate(porcentaje_nulos.values):
    plt.text(v + 0.01, i, f"{v:.2f}%", va='center')

minv, maxv = porcentaje_nulos.min(), porcentaje_nulos.max()
plt.xlim(minv - 0.05, maxv + 0.05)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Chequeo de duplicados
duplicados = df.duplicated().sum()
print(f"Cantidad de registros duplicados: {duplicados}")

# Estadísticos descriptivos y rango
variables_numericas = ['balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
stats = df[variables_numericas].describe().T
stats["rango"] = stats["max"] - stats["min"]
display(stats)

# Sesgo (skewness)
sesgo = df[variables_numericas].skew().sort_values(ascending=False)
print("Sesgo (skewness) de las variables numéricas:")
display(sesgo)

# Outliers Qi
Q1 = df[variables_numericas].quantile(0.25)
Q3 = df[variables_numericas].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df[variables_numericas] < (Q1 - 1.5 * IQR)) | (df[variables_numericas] > (Q3 + 1.5 * IQR))).sum()
print("Cantidad de outliers por variable:")
display(outliers)

# Boxplots para variables numéricas

# --- Balance ---
plt.figure(figsize=(6,4))

sns.boxplot(x=np.log1p(df['balance'] - df['balance'].min() + 1), color=color_base)

plt.title('Distribución de la variable Balance (escala logarítmica)', fontsize=11)
plt.xlabel('log(Balance)', fontsize=10)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# --- Day ---
plt.figure(figsize=(6,4))
sns.boxplot(x=df['day'], color=color_base)
plt.title('Distribución de la variable Day', fontsize=11)
plt.xlabel('Día del mes', fontsize=10)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- Duration ---
plt.figure(figsize=(6,4))
sns.boxplot(x=df['duration'], color=color_base)
plt.title('Distribución de la variable Duration', fontsize=11)
plt.xlabel('Duración de la llamada (segundos)', fontsize=10)
plt.xlim(0, 1200)  # te muestra la mayoría de llamadas
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- Campaign ---
plt.figure(figsize=(6,4))
sns.boxplot(x=df['campaign'], color=color_base)
plt.title('Distribución de la variable Campaign', fontsize=11)
plt.xlabel('Cantidad de contactos en la campaña', fontsize=10)
plt.xlim(0, 15)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- Pdays ---
plt.figure(figsize=(6,4))
sns.boxplot(x=df['pdays'], color=color_base)
plt.title('Distribución de la variable Pdays', fontsize=11)
plt.xlabel('Días desde el último contacto', fontsize=10)
plt.xlim(0, 400)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- Previous ---
plt.figure(figsize=(6,4))
sns.boxplot(x=df['previous'], color=color_base)
plt.title('Distribución de la variable Previous', fontsize=11)
plt.xlabel('Cantidad de contactos previos', fontsize=10)
plt.xlim(0, 10)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


profile = ProfileReport(df, title="Tema_4", explorative=True)
profile.to_file("Tema_4.html")

archivo_html = os.path.abspath("Tema_4.html")
webbrowser.open(f"file://{archivo_html}")

variables_binarias = ['default', 'housing', 'loan', 'y']
variables_categoricas = ['job', 'marital', 'education', 'contact', 'poutcome']

print("-------------------- DISTRIBUCIÓN DE VARIABLES NUMÉRICAS CONTÍNUAS --------------------")

# --- BALANCE ---
plt.figure(figsize=(6,4))
sns.histplot(df['balance'], bins=50, color=color_base, edgecolor='black', kde=False)
plt.yscale('log')
plt.title('Distribución de la variable Balance', fontsize=11)
plt.xlabel('Balance', fontsize=10)
plt.ylabel('Frecuencia (escala log)', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- DAY ---
plt.figure(figsize=(6,4))
sns.histplot(df['day'], bins=31, color=color_base, edgecolor='black', kde=False)
plt.xticks(range(1,32,2))
plt.title('Distribución de la variable Day', fontsize=11)
plt.xlabel('Day', fontsize=10)
plt.ylabel('Frecuencia', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- DURATION ---
plt.figure(figsize=(6,4))
sns.histplot(df['duration'], bins=40, color=color_base, edgecolor='black', kde=False)
plt.xlim(0, 1200)
plt.title('Distribución de la variable Duration (limitada a 1200s)', fontsize=11)
plt.xlabel('Duration (segundos)', fontsize=10)
plt.ylabel('Frecuencia', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- CAMPAIGN ---
plt.figure(figsize=(6,4))
sns.histplot(df['campaign'], bins=30, color=color_base, edgecolor='black', kde=False)
plt.yscale('log')
plt.title('Distribución de la variable Campaign', fontsize=11)
plt.xlabel('Campaign', fontsize=10)
plt.ylabel('Frecuencia (escala log)', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- PDAYS ---
plt.figure(figsize=(6,4))
sns.histplot(df['pdays'], bins=50, color=color_base, edgecolor='black', kde=False)
plt.xlim(0, 400)
plt.yscale('log')
plt.title('Distribución de la variable Pdays', fontsize=11)
plt.xlabel('Pdays', fontsize=10)
plt.ylabel('Frecuencia (escala log)', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# --- PREVIOUS ---
plt.figure(figsize=(6,4))
sns.histplot(df['previous'], bins=20, color=color_base, edgecolor='black', kde=False)
plt.xlim(0, 10)
plt.yscale('log')
plt.title('Distribución de la variable Previous', fontsize=11)
plt.xlabel('Previous', fontsize=10)
plt.ylabel('Frecuencia (escala log)', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

print("-------------------- DISTRIBUCIÓN DE VARIABLES BINARIAS --------------------")
for col in variables_binarias:
    plt.figure(figsize=(5,3))
    valores = df[col].value_counts(dropna=False)

    sns.barplot(
        x=valores.index.astype(str),
        y=valores.values,
        color=color_base,
        edgecolor='black'
    )
    
    plt.ylim(0, valores.max() * 1.15)
    plt.title(f'Distribución de la variable {col.capitalize()}', fontsize=11)
    plt.xlabel(col.capitalize(), fontsize=10)
    plt.ylabel('Frecuencia', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.6)

    for i, v in enumerate(valores.values):
        plt.text(i, v + (valores.max() * 0.02), str(v), ha='center', fontsize=9, color=colores['oscuro'])
    
    plt.tight_layout()
    plt.show()

print("-------------------- DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS --------------------")
for col in variables_categoricas:
    plt.figure(figsize=(8,4))
    valores = df[col].value_counts(dropna=False).sort_values(ascending=False)
    
    sns.barplot(
        x=valores.index.astype(str),
        y=valores.values,
        color=color_base,
        edgecolor='black'
    )

    plt.ylim(0, valores.max() * 1.20)
    plt.title(f'Distribución de la variable {col.capitalize()}', fontsize=11)
    plt.xlabel(col.capitalize(), fontsize=10)
    plt.ylabel('Frecuencia', fontsize=10)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.6)

    for i, v in enumerate(valores.values):
        plt.text(i, v + (valores.max() * 0.02), str(v), ha='center', fontsize=9, color=colores['oscuro'])
    
    plt.tight_layout()
    plt.show()

# Limpieza de datos e imputaciones

### Duplicados

In [ ]:
df_cleaned = df.copy()

dup = df_cleaned.duplicated().sum()
print(f"Duplicados encontrados: {dup}")

if dup > 0:
    df_cleaned = df_cleaned.drop_duplicates()
    print(f"Duplicados eliminados. Nuevo total: {df_cleaned.shape[0]} filas.")
else:
    print("No se encontraron duplicados.")
    


### Corrección tipo de datos

In [ ]:
df_cleaned.replace(
    ['unknown', 'unknown_age', 'nonexistent', 'NA', 'na', 'None'],
    np.nan,
    inplace=True
)

# Asegurar tipos numéricos
num_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
for col in num_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

print("Tipos de datos corregidos y valores especiales convertidos a NaN.")

### Inconsistencias y relaciones lógicas

In [ ]:
# Valores especiales en pdays
df_cleaned['pdays'] = df_cleaned['pdays'].replace(-1, np.nan)

# Relación previous - pdays - poutcome
df_cleaned.loc[df_cleaned['previous'] == 0, 'pdays'] = np.nan
df_cleaned.loc[df_cleaned['previous'] == 0, 'poutcome'] = 'nonexistent'

# duration = 0 entonces y = 'no'
df_cleaned.loc[df_cleaned['duration'] == 0, 'y'] = 'no'

# Rangos válidos
df_cleaned.loc[(df_cleaned['age'] < 18) | (df_cleaned['age'] > 100), 'age'] = np.nan
df_cleaned.loc[(df_cleaned['day'] < 1) | (df_cleaned['day'] > 31), 'day'] = np.nan

# Campañas y contactos negativos - NaN
for col in ['campaign', 'previous']:
    df_cleaned.loc[df_cleaned[col] < 0, col] = np.nan

print("Inconsistencias y relaciones lógicas corregidas.")

### Limpieza por variable 

In [ ]:
# --- AGE ---
df_cleaned = df_cleaned[(df_cleaned['age'].between(18, 100)) | (df_cleaned['age'].isna())]
mediana_edad = df_cleaned['age'].median()
df_cleaned['age'] = df_cleaned['age'].fillna(mediana_edad)

# --- JOB ---
df_cleaned['job'] = (
    df_cleaned['job'].astype(str).str.strip().str.lower()
    .replace({'admin.': 'admin', '12345': 'unknown', 'nan': 'unknown', 'none': 'unknown'})
    .fillna('unknown')
)

# --- MARITAL ---
df_cleaned['marital'] = (
    df_cleaned['marital'].astype(str).str.strip().str.lower()
    .replace({'nan': 'unknown', 'none': 'unknown'})
    .fillna('unknown')
)

# --- EDUCATION ---
df_cleaned['education'] = (
    df_cleaned['education'].astype(str).str.strip().str.lower()
    .replace({
        'basic.4y': 'primary', 'basic.6y': 'primary',
        'basic.9y': 'secondary', 'university.degree': 'tertiary',
        'high.school': 'secondary', 'illiterate': 'primary',
        'nan': 'unknown', 'none': 'unknown'
    })
    .fillna('unknown')
)

# Imputación contextual según job
imputacion_por_job = {
    'management': 'tertiary', 'technician': 'tertiary',
    'entrepreneur': 'tertiary', 'self-employed': 'tertiary',
    'admin': 'secondary', 'services': 'secondary',
    'blue-collar': 'primary', 'housemaid': 'primary',
    'unemployed': 'primary', 'student': 'tertiary',
    'retired': 'unknown', 'unknown': 'unknown'
}
mask = (df_cleaned['education'] == 'unknown') & (df_cleaned['job'].notna())
df_cleaned.loc[mask, 'education'] = df_cleaned.loc[mask, 'job'].map(imputacion_por_job)

# --- DEFAULT / HOUSING / LOAN ---
for col in ['default', 'housing', 'loan']:
    df_cleaned[col] = (
        df_cleaned[col].astype(str).str.strip().str.lower()
        .replace({'yes': True, 'no': False, 'nan': np.nan, 'none': np.nan})
        .astype('boolean')
    )

# --- DAY ---
df_cleaned = df_cleaned[(df_cleaned['day'] >= 1) & (df_cleaned['day'] <= 31)]
df_cleaned['day'] = df_cleaned['day'].astype('Int64')

# --- CAMPAIGN ---
q99 = df_cleaned['campaign'].quantile(0.99)
df_cleaned.loc[df_cleaned['campaign'] > q99, 'campaign'] = np.nan
df_cleaned['campaign'] = df_cleaned['campaign'].fillna(df_cleaned['campaign'].median()).astype('Int64')

# --- PREVIOUS ---
q99 = df_cleaned['previous'].quantile(0.99)
df_cleaned.loc[df_cleaned['previous'] > q99, 'previous'] = np.nan
df_cleaned['previous'] = df_cleaned['previous'].astype('Int64')

# --- PDAYS ---
df_cleaned.loc[df_cleaned['pdays'] == -1, 'pdays'] = np.nan
df_cleaned['pdays'] = df_cleaned['pdays'].astype('Int64')

# --- POUTCOME ---
df_cleaned['poutcome'] = (
    df_cleaned['poutcome'].astype(str).str.strip().str.lower()
    .replace({
        'nonexistant': 'nonexistent', 'unknown': 'nonexistent',
        'none': 'nonexistent', 'nan': np.nan
    })
)
df_cleaned['poutcome'] = pd.Categorical(
    df_cleaned['poutcome'], categories=['failure', 'nonexistent', 'success'], ordered=False
)

# --- Y (variable objetivo) ---
df_cleaned['y'] = (
    df_cleaned['y'].astype(str).str.strip().str.lower()
    .replace({'yes': True, 'no': False, 'nan': np.nan, 'none': np.nan})
    .astype('boolean')
)
df_cleaned.loc[df_cleaned['duration'] == 0, 'y'] = False

### Columnas constantes / redundantes

In [ ]:
# Eliminar columnas con un solo valor
constantes = [col for col in df_cleaned.columns if df_cleaned[col].nunique(dropna=False) <= 1]
if constantes:
    df_cleaned.drop(columns=constantes, inplace=True)
    print(f"Columnas constantes eliminadas: {constantes}")

# Eliminar redundantes
if 'duration' in df_cleaned.columns:
    df_cleaned.drop(columns=['duration'], inplace=True)
    print("duration eliminada (solo conocida post-contacto).")

if 'pdays' in df_cleaned.columns:
    df_cleaned.drop(columns=['pdays'], inplace=True)
    print("pdays eliminada (alta correlación con previous).")

print(f"Columnas restantes: {df_cleaned.shape[1]}")

### Imputación de nulos

In [ ]:
# Numéricas con sesgos - mediana
cols_mediana = ['balance', 'campaign', 'previous']
imputer_med = SimpleImputer(strategy='median')
df_cleaned[cols_mediana] = imputer_med.fit_transform(df_cleaned[cols_mediana])

# Numéricas simétricas - media
cols_media = ['age', 'day']
imputer_mean = SimpleImputer(strategy='mean')
df_cleaned[cols_media] = imputer_mean.fit_transform(df_cleaned[cols_media])

# Categóricas - moda
cols_moda = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
imputer_mode = SimpleImputer(strategy='most_frequent')
df_cleaned[cols_moda] = imputer_mode.fit_transform(df_cleaned[cols_moda])

# Binarias - completar con valor más frecuente
cols_bin = ['default', 'housing', 'loan']
for col in cols_bin:
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

print("\nImputación final completada. Resumen de nulos:")
print(df_cleaned.isna().sum())

### Verificación

In [ ]:
print(f"Dataset final: {df_cleaned.shape[0]} filas y {df_cleaned.shape[1]} columnas")
print("Columnas finales:")
print(df_cleaned.columns.tolist())

print("\nCantidad total de valores nulos:", df_cleaned.isna().sum().sum())